In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error 

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv("/home/sky/Documents/yash/forest fire detection/forestfires.csv")
data 

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [3]:
data = data.drop(["month","day"], axis=1)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   FFMC    517 non-null    float64
 3   DMC     517 non-null    float64
 4   DC      517 non-null    float64
 5   ISI     517 non-null    float64
 6   temp    517 non-null    float64
 7   RH      517 non-null    int64  
 8   wind    517 non-null    float64
 9   rain    517 non-null    float64
 10  area    517 non-null    float64
dtypes: float64(8), int64(3)
memory usage: 44.6 KB


In [5]:
def preprocess_inputs(df, task='regression'):
    df = df.copy()
    
    # split df into X and y
    if task == 'regression':
        y = df['area']
    elif task == 'classification':
        y = df['area'].apply(lambda x: 1 if x > 0 else 0) # will create a new col that will suggest if it is a large fire or small fire
    
    X = df.drop('area', axis=1) # all other data
    
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, shuffle=True, random_state=1)
    
    # scaler feature data (X)
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
        
    return X_train, X_test, y_train, y_test

In [11]:
# Regression

In [34]:
X_train_R, X_test_R, y_train_R, y_test_R = preprocess_inputs(data, task='regression')

In [63]:
svr = SVR(kernel='rbf')
svr.fit(X_train_R, y_train_R)
print("SVR R^2: {:.5f}".format(svr.score(X_test_R, y_test_R)))

y_pred_R = svr.predict(X_test_R)
print("MSE : ",mean_squared_error(y_test_R, y_pred_R))

SVR R^2: -0.04467
MSE :  7868.441915678149


In [64]:
## Prediction on 10 Values
area = np.round(y_pred_R[:10])

In [65]:
for p in area[:10]:
    print("Area of fire : ",p)

Area of fire :  1.0
Area of fire :  1.0
Area of fire :  1.0
Area of fire :  1.0
Area of fire :  1.0
Area of fire :  0.0
Area of fire :  1.0
Area of fire :  2.0
Area of fire :  1.0
Area of fire :  -0.0


In [12]:
#Classification

In [14]:
X_train_C, X_test_C, y_train_C, y_test_C = preprocess_inputs(data, task='classification')

In [15]:
svm = SVC(kernel='rbf').fit(X_train_C, y_train_C)
print('Accuracy of SVC classifier on training set: {:.2f}'
     .format(svm.score(X_train_C, y_train_C)))
print('Accuracy of SVC classifier on test set: {:.2f}'
     .format(svm.score(X_test_C, y_test_C)))

Accuracy of SVC classifier on training set: 0.70
Accuracy of SVC classifier on test set: 0.47


In [16]:
## Prediction on 10 Values
pred = svm.predict(X_test_C)
pred[:10]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [25]:

for p in pred[:10]:
    if p == 1:
        print(p," : There is Fire")
    else:
        print(p," : There is no fire")

1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no file
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no file


In [38]:
svm_poly=SVC(kernel='poly', C=100,probability=True).fit(X_train_C, y_train_C)
print('Accuracy of SVC Poly classifier on training set: {:.2f}'
     .format(svm_poly.score(X_train_C, y_train_C)))
print('Accuracy of SVC Poly classifier on test set: {:.2f}'
     .format(svm_poly.score(X_test_C, y_test_C)))

Accuracy of SVC Poly classifier on training set: 0.79
Accuracy of SVC Poly classifier on test set: 0.49


In [46]:
## Prediction on 10 Values
pred = svm_poly.predict(X_test_C)
pred[:10]

array([1, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [47]:
for p in pred[:10]:
    if p == 1:
        print(p," : There is Fire")
    else:
        print(p," : There is no fire")

1  : There is Fire
1  : There is Fire
0  : There is no fire
1  : There is Fire
0  : There is no fire
1  : There is Fire
0  : There is no fire
1  : There is Fire
1  : There is Fire
1  : There is Fire


In [48]:
svm_sigmoid=SVC(kernel='sigmoid', C=100,probability=True).fit(X_train_C, y_train_C)
print('Accuracy of SVC Sigmoid classifier on training set: {:.2f}'
     .format(svm_sigmoid.score(X_train_C, y_train_C)))
print('Accuracy of SVC Sigmoid classifier on test set: {:.2f}'
     .format(svm_sigmoid.score(X_test_C, y_test_C)))

Accuracy of SVC Sigmoid classifier on training set: 0.51
Accuracy of SVC Sigmoid classifier on test set: 0.33


In [49]:
## Prediction on 10 Values
pred = svm_sigmoid.predict(X_test_C)
pred[:10]

array([1, 1, 1, 1, 0, 0, 1, 1, 1, 0])

In [50]:

for p in pred[:10]:
    if p == 1:
        print(p," : There is Fire")
    else:
        print(p," : There is no fire")

1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire
0  : There is no fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire


In [51]:
svm_decf = SVC(kernel='rbf', C=1,decision_function_shape='ovr').fit(X_train_C, y_train_C)
print('Accuracy of SVC decision_function classifier on training set: {:.2f}'
     .format(svm_decf.score(X_train_C, y_train_C)))
print('Accuracy of SVC decision_function classifier on test set: {:.2f}'
     .format(svm_decf.score(X_test_C, y_test_C)))


Accuracy of SVC decision_function classifier on training set: 0.70
Accuracy of SVC decision_function classifier on test set: 0.47


In [52]:
## Prediction on 10 Values
pred = svm_decf.predict(X_test_C)
pred[:10]

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [53]:
for p in pred[:10]:
    if p == 1:
        print(p," : There is Fire")
    else:
        print(p," : There is no fire")

1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire


In [54]:
linear_svc = LinearSVC().fit(X_train_C, y_train_C)

print('Accuracy of Linear SVC classifier on training set: {:.2f}'
     .format(linear_svc.score(X_train_C, y_train_C)))
print('Accuracy of Linear SVC classifier on test set: {:.2f}'
     .format(linear_svc.score(X_test_C, y_test_C)))


Accuracy of Linear SVC classifier on training set: 0.59
Accuracy of Linear SVC classifier on test set: 0.54


In [55]:
## Prediction on 10 Values
pred = linear_svc.predict(X_test_C)
pred[:10]

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 0])

In [56]:
for p in pred[:10]:
    if p == 1:
        print(p," : There is Fire")
    else:
        print(p," : There is no fire")

0  : There is no fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire
1  : There is Fire
1  : There is Fire
1  : There is Fire
0  : There is no fire
